In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.patches as patches
import csv as csv

In [ ]:
#!pip install import_ipynb

In [20]:
import Metrica_IO as mio
import Metrica_Viz as mviz

In [21]:
def to_single_playing_direction(home,away,events):
    '''
    Flip coordinates in second half so that each team always shoots in the same direction through the match.
    '''
    for team in [home,away,events]:
        if (team.Period == 2).any():
          second_half_idx = team[team['Period'] == 2].index[0]

          # Select columns ending with 'x' or 'y' (case-insensitive)
          columns = [c for c in team.columns if c[-1].lower() in ['x', 'y']]

          # Flip the coordinates for the second half
          team.loc[second_half_idx:, columns] *= -1
    return home,away,events

In [ ]:
DATADIR = '/content/drive/MyDrive/SportsData/'
game_id = 2

events = mio.read_event_data(DATADIR,game_id)

tracking_home = mio.tracking_data(DATADIR,game_id,'Home')
tracking_away = mio.tracking_data(DATADIR,game_id,'Away')

tracking_home = mio.to_metric_coordinates(tracking_home)
tracking_away = mio.to_metric_coordinates(tracking_away)

events = mio.to_metric_coordinates(events)

# 전반, 후반 방향 일치 (후반전 선수 방향 filp)
tracking_home, tracking_away, events = to_single_playing_direction(tracking_home, tracking_away, events)

Reading team: home
Reading team: away


In [ ]:
PLOTDIR = '/content/drive/MyDrive/SportsData/'
mviz.save_match_clip(tracking_home.iloc[73600:73600+500],tracking_away.iloc[73600:73600+500],PLOTDIR,fname='home_goal_2',include_player_velocities=False)

Generating movie...done


In [ ]:
DATADIR2 = './interpolation'
home_list = []
away_list = []

for i in range(1, 12):
    df = pd.read_csv(f'{DATADIR2}/home_{i}.csv')
    home_list.append(df)

for i in range(1, 12):
    df = pd.read_csv(f'{DATADIR2}/away_{i}.csv')
    away_list.append(df)

home_list[0].head()

,frame,x,y,time
0,0,0.914773,0.504444,0.00
1,1,0.914773,0.504444,0.04
2,2,0.914773,0.504444,0.08
3,3,0.916193,0.504444,0.12
4,4,0.917614,0.504444,0.16


In [ ]:
for h in home_list:
    h['period'] = 1
for a in away_list:
    a['period'] = 1

In [ ]:
"""
df_6['period'] = (df_6['time'] > 2696.48).astype(int) + 1
df_7['period'] = (df_7['time'] > 2696.48).astype(int) + 1
df_4['period'] = (df_4['time'] > 2696.48).astype(int) + 1
"""

In [ ]:
for i in range(1, 12):
    df = pd.read_csv(f'{DATADIR2}/home_{i}.csv')
    home_list.append(df)

for i in range(1,12):
    home_list[i-1] = home_list[i-1].rename(columns={f'x': 'Home_{i}_x', 'y': 'Home_{i}_y'})
    away_list[i-1] = away_list[i-1].rename(columns={f'x': 'Away_{i}_x', 'y': 'Away_{i}_y'})

home_merged = home_list[0][['frame', 'period', 'time', 'Home_1_x', 'Home_1_y']]  # 데이터 병합 (frame, period, time)
away_merged = away_list[0][['frame', 'period', 'time', 'Away_1_x', 'Away_1_y']]

for i in range(2, 12):
    home_merged = home_merged.merge(
        home_list[i-1][['frame', f'Home_{i}_x', f'Home_{i}_y']],
        on='frame'
    )
    away_merged = away_merged.merge(
        away_list[i-1][['frame', f'Away_{i}_x', f'Away_{i}_y']],
        on='frame'
    )

# Frame을 인덱스로 설정
home_merged = home_merged.set_index('frame')
home_merged.index.name = 'Frame'
away_merged = away_merged.set_index('frame')
away_merged.index.name = 'Frame'
print(home_merged)
print(away_merged)

       period   time  Home_1_x  Home_1_y  Home_2_x  Home_2_y  Home_3_x  \
Frame                                                                    
0           1   0.00  0.914773  0.504444  0.610795  0.497778  0.509943   
1           1   0.04  0.914773  0.504444  0.610795  0.500000  0.509943   
2           1   0.08  0.914773  0.504444  0.611269  0.500000  0.509943   
3           1   0.12  0.916193  0.504444  0.611742  0.500000  0.509943   
4           1   0.16  0.917614  0.504444  0.612216  0.500000  0.509943   
...       ...    ...       ...       ...       ...       ...       ...   
548         1  21.92  0.914418  0.553333  0.600852  0.675556  0.427557   
549         1  21.96  0.914773  0.553333  0.600852  0.673333  0.427557   
550         1  22.00  0.914773  0.555556  0.600852  0.673333  0.427557   
551         1  22.04  0.914773  0.553333  0.600852  0.673333  0.426705   
552         1  22.08  0.914773  0.551111  0.600852  0.673333  0.425852   

       Home_3_y  Home_4_x  Home_4_y  

In [ ]:
home_merged['ball_x'] = 0
home_merged['ball_y'] = 0
away_merged['ball_x'] = 0
away_merged['ball_y'] = 0

In [31]:
home_merged = home_merged.rename(columns={'period': 'Period', 'time': 'Time [s]'})
away_merged = away_merged.rename(columns={'period': 'Period', 'time': 'Time [s]'})

In [32]:
away_merged.head()

,Period,Time [s],Away_1_x,Away_1_y,Away_2_x,Away_2_y,Away_3_x,Away_3_y,Away_4_x,Away_4_y,...,Away_8_x,Away_8_y,Away_9_x,Away_9_y,Away_10_x,Away_10_y,Away_11_x,Away_11_y,ball_x,ball_y
Frame,,,,,,,,,,,,,,,,,,,,,
0,1,0.00,0.117898,0.504444,0.488636,0.822222,0.372159,0.797778,0.497159,0.500000,...,0.382102,0.571111,0.386364,0.422222,0.419034,0.593333,0.490057,0.188889,0,0
1,1,0.04,0.117898,0.504444,0.488636,0.822222,0.372159,0.797778,0.500000,0.504444,...,0.382102,0.568889,0.387784,0.422222,0.420455,0.593333,0.490057,0.188889,0,0
2,1,0.08,0.117898,0.504444,0.488636,0.822222,0.372159,0.797778,0.500000,0.502222,...,0.382102,0.571111,0.387784,0.422222,0.420455,0.593333,0.490057,0.188889,0,0
3,1,0.12,0.117898,0.504444,0.488636,0.822222,0.372159,0.797778,0.500000,0.504444,...,0.382102,0.571111,0.386364,0.424444,0.419034,0.593333,0.490057,0.188889,0,0
4,1,0.16,0.117898,0.504444,0.488636,0.822222,0.372869,0.797778,0.500000,0.502222,...,0.379261,0.577778,0.386364,0.424444,0.420455,0.593333,0.490057,0.188889,0,0


In [33]:
home_merged.index

Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       543, 544, 545, 546, 547, 548, 549, 550, 551, 552],
      dtype='int64', name='Frame', length=553)

In [34]:
away_merged.index

Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       541, 542, 543, 544, 545, 546, 547, 548, 549, 550],
      dtype='int64', name='Frame', length=551)

In [35]:
away_merged.shape

(551, 26)

In [36]:

home_merged = home_merged.iloc[:551]
home_merged.shape

(551, 26)

In [37]:
df_home = mio.to_metric_coordinates(home_merged)
df_away = mio.to_metric_coordinates(away_merged)


In [38]:
df_away[8:12]

,Period,Time [s],Away_1_x,Away_1_y,Away_2_x,Away_2_y,Away_3_x,Away_3_y,Away_4_x,Away_4_y,...,Away_8_x,Away_8_y,Away_9_x,Away_9_y,Away_10_x,Away_10_y,Away_11_x,Away_11_y,ball_x,ball_y
Frame,,,,,,,,,,,,,,,,,,,,,
8,1,0.32,-40.502841,-0.302222,-1.355114,-22.062222,-13.551136,-20.248889,0.150568,-0.604444,...,-12.497159,-4.986667,-11.894886,5.137778,-8.431818,-6.346667,-1.053977,21.155556,-53.0,34.0
9,1,0.36,-40.502841,-0.302222,-1.355114,-22.213333,-13.099432,-20.551111,0.150568,-0.604444,...,-12.497159,-5.137778,-11.894886,4.835556,-8.582386,-6.346667,-1.053977,21.457778,-53.0,34.0
10,1,0.40,-40.502841,-0.302222,-1.355114,-22.364444,-13.099432,-20.551111,0.150568,-0.604444,...,-12.497159,-5.137778,-11.894886,4.835556,-8.582386,-6.497778,-0.903409,21.155556,-53.0,34.0
11,1,0.44,-40.502841,-0.302222,-1.204545,-22.062222,-13.099432,-20.551111,0.150568,-0.604444,...,-12.497159,-5.137778,-11.894886,4.835556,-8.582386,-6.346667,-0.903409,21.457778,-53.0,34.0


In [39]:
df_home.to_csv('./results/home.csv', index=False)

In [40]:
df_away.to_csv('./results/away.csv', index=False)

In [ ]:
#PLOTDIR = './results'
mviz.save_match_clip(df_home.iloc[0:0+500],df_away.iloc[0:0+500],PLOTDIR,fname='result_1',include_player_velocities=False)

RuntimeError: Requested MovieWriter (ffmpeg) not available